In [1]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table

%matplotlib inline
import matplotlib.pyplot as plt

import pygtc
import pandas as pd

from getdist import plots
from getdist.mcsamples import  MCSamples
import getdist
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))

from IPython.display import display, Math

from richness_mass_calib import create_richness_mass_calib

GetDist Version: 1.4.5, Matplotlib version: 3.7.2


In [2]:
# #Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [3]:
# Data
DC2_halos_m200c = fits.open(
    "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
)
dt_halos = Table(DC2_halos_m200c[1].data)
# dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']

In [4]:
rmdata = create_richness_mass_calib(dt_halos)
lnrich_ext = Nc.ClusterMassLnrichExt()

lnrich_ext.param_set_by_name("mu", 3.95849990149314)
lnrich_ext.param_set_by_name("muM1", 0.834834799745832)
lnrich_ext.param_set_by_name("muZ1", 0.695004688438083)
lnrich_ext.param_set_by_name("muM2", 0.0421606025663114)
lnrich_ext.param_set_by_name("muZ2", -0.051715032078255)
lnrich_ext.param_set_by_name("muMZ", 0.0402769634041103)
        

lnrich_ext.param_set_by_name("sigma0", 0.306313313048325)
lnrich_ext.param_set_by_name("sigmaM1", 0.00737198200713169)
lnrich_ext.param_set_by_name("sigmaZ1", -0.0212988049057424)
lnrich_ext.param_set_by_name("sigmaM2", 0.015610993686449)
lnrich_ext.param_set_by_name("sigmaZ2", 0.00480098229907118)
lnrich_ext.param_set_by_name("sigmaMZ", 0.0111750633563743)

lnrich_ext.param_set_by_name("cut", 1e15)
lnrich_ext.param_set_by_name("cutM1", 0)
lnrich_ext.param_set_by_name("cutZ1", 0)

mset = Ncm.MSet()
mset.set(lnrich_ext)

rmdata.m2lnL_val(mset)

119436.72488745325

In [5]:
dset = Ncm.Dataset.new()
dset.append_data(rmdata)
lh = Ncm.Likelihood.new(dset)

mset.param_set_all_ftype(Ncm.ParamType.FREE)
mset.param_set_ftype(7000, 12, Ncm.ParamType.FIXED)
mset.param_set_ftype(7000, 13, Ncm.ParamType.FIXED)
mset.param_set_ftype(7000, 14, Ncm.ParamType.FIXED)
mset.prepare_fparam_map()

fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL)
fit.log_info()
fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)


#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.95849990149314    [FREE]
#   -    muM1[01]:  0.834834799745832   [FREE]
#   -    muZ1[02]:  0.695004688438083   [FREE]
#   -    muM2[03]:  0.0421606025663114  [FREE]
#   -    muZ2[04]: -0.051715032078255   [FREE]
#   -    muMZ[05]:  0.0402769634041103  [FREE]
#   -  sigma0[06]:  0.306313313048325   [FREE]
#   - sigmaM1[07]:  0.00737198200713169 [FREE]
#   - sigmaZ1[08]: -0.0212988049057424  [FREE]
#   - sigmaM2[09]:  0.015610993686449   [FREE]
#   - sigmaZ2[10]:  0.00480098229907118 [FREE]
#   - sigmaMZ[11]:  0.0111750633563743  [FREE]
#   -     cut[12]:  1e+15               [FIX

In [6]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.95845743927829    [FREE]
#   -    muM1[01]:  0.834828063823967   [FREE]
#   -    muZ1[02]:  0.695035726088091   [FREE]
#   -    muM2[03]:  0.0421630143455661  [FREE]
#   -    muZ2[04]: -0.0517117425402657  [FREE]
#   -    muMZ[05]:  0.0402889248541696  [FREE]
#   -  sigma0[06]:  0.30629461997396    [FREE]
#   - sigmaM1[07]:  0.00737598847048528 [FREE]
#   - sigmaZ1[08]: -0.0212926821468329  [FREE]
#   - sigmaM2[09]:  0.015611864636027   [FREE]
#   - sigmaZ2[10]:  0.00479784520418163 [FREE]
#   - sigmaMZ[11]:  0.01117065132677    [FREE]
#   -     cut[12]:  1e+15               [FIX

In [7]:
nwalkers = 1000

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("model_extz2.fits")

esmcmc.start_run()
esmcmc.run(2000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 1000.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.95842427513048    [FREE]
#   -    muM1[01

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



   -0.0191818822085792     0.0166565946786507    0.00453035838601374     0.0112455907538647
# NcmFitESMCMC: Catalog row 1999151: m2lnL =     119452.393874885, recalculated to     119452.393874885, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 1999151 values:      119452.393874885       3.97027640466887      0.839193547026899      0.692420574332234     0.0418130649972752    -0.0526556435731685      0.037968930456289      0.308062166608116      0.011041025561516      -0.01941395879201       0.01654411542074    0.00487944984413996     0.0118226866186621
# NcmFitESMCMC: Catalog row 1999152: m2lnL =     119451.601684279, recalculated to     119451.601684279, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 1999152 values:      119451.601684279       3.95644956434634       0.83437068558752      0.694235226613441     0.0418553256185388    -0.0524985130814048     0.0389122112628867      0.311562276529162     0.0110160346142493    -0.0235135655138691     0.0161396610572521    0.004551848

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



 4.3575e-07   8.3381e-07   3.5246e-06   2.4043e-06   1.7614e-06   5.6088e-07   3.643e-07    6.4608e-07 
# NcmMSetCatalog: Current sd:     23686        0.0083328    0.0053897    0.0033681    0.001166     0.00061624   0.0011792    0.0049846    0.0034002    0.002491     0.00079321   0.0005152    0.0009137  
# NcmMSetCatalog: Current var:    5.6103e+08   6.9435e-05   2.9049e-05   1.1344e-05   1.3595e-06   3.7975e-07   1.3905e-06   2.4846e-05   1.1562e-05   6.2052e-06   6.2918e-07   2.6543e-07   8.3484e-07 
# NcmMSetCatalog: Current tau:    1            1            1            1            1            1            1            1            1            1            1            1            1          
#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   -------------------------------------------------------------------------------------------------------------------

In [ ]:
esmcmc.mean_covar()
fit.log_covar()

#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#      mu[07000:00] =  3.959       +/-  0.008333    |  1           |  0.7712      | -0.3783      |  0.5187      |  0.0366      | -0.2733      |  0.1168      |  0.2581      |  0.04415     |  0.2644      | -0.0003456   |  0.03162     |
#    muM1[07000:01] =  0.8355      +/-  0.00539     |  0.7712      |  1           |  0.06498     |  0.8778      | -0.07688     |  0.01352     |  0.1707      |  0.3522      |  0.07643     |  0.3439      | -0.04509     |  0.02428     |
#    muZ1[07000:02] =  0.6949      +/-  0.003368    | -0.3783      |  0.06498     |  1           |  0.2777      | -0.2402      |  0.6661      |  0.05198     |  0.03933     |  

In [ ]:
fit.log_covar()

In [ ]:
fit.log_info()

In [ ]:
data_fit_full = pd.DataFrame(fits.open("model_extz2.fits")[1].data).iloc[:, 1:13].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12',
]
labels=["\mu_0", "\mu_{M1}", "\mu_{M2}", "\mu_{Z1}"," \mu_{Z2}", "\mu_{MZ}", "\sigma_0", "\sigma_{M1}", "\sigma_{M2}", "\sigma_{Z1}", "\sigma_{Z2}", "\sigma_{MZ}" ]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples2 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples2.removeBurn(0.3)


In [ ]:
# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples2],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['orange'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'orange'}],
    legend_labels=["Estendido", ],
)



plt.show()
display(Math(samples2.getInlineLatex('1',limit=1)))